In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix

In [2]:
# CSV-Datei laden
df = pd.read_csv("data_sorted_vibration.csv")  # ← Datei ggf. anpassen

# Zeitreihe als Liste interpretieren
df["oscillation"] = df["oscilation"].apply(lambda x: np.array(eval(x), dtype=float))

# Statistische Merkmale extrahieren
df["osc_mean"] = df["oscillation"].apply(np.mean)
df["osc_std"] = df["oscillation"].apply(np.std)
df["osc_max"] = df["oscillation"].apply(np.max)
df["osc_min"] = df["oscillation"].apply(np.min)


In [3]:
# Features & Ziel
X = df[["osc_mean", "osc_std", "osc_max", "osc_min"]]
y = df["is cracked"]

# Trainings-/Testdaten splitten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modell trainieren
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Vorhersage machen
y_pred = model.predict(X_test)


In [4]:
# Ergebnisse ausgeben
print(" Klassifikations-Report:")
print(classification_report(y_test, y_pred))

print(" Konfusionsmatrix:")
print(confusion_matrix(y_test, y_pred))

f1 = f1_score(y_test, y_pred)
print(f"\n F1-Score (Testdaten): {f1:.4f}")


 Klassifikations-Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.97        14
           1       0.00      0.00      0.00         1

    accuracy                           0.93        15
   macro avg       0.47      0.50      0.48        15
weighted avg       0.87      0.93      0.90        15

 Konfusionsmatrix:
[[14  0]
 [ 1  0]]

 F1-Score (Testdaten): 0.0000


c:\Users\User\Desktop\Automatisierungstechnik\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\Desktop\Automatisierungstechnik\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\Desktop\Automatisierungstechnik\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod

## Analyse des Klassifikationsergebnisses

Die Drop-Vibration-Daten wurden zur Klassifikation defekter Flaschen genutzt. Die Zielvariable `is cracked` zeigt an, ob eine Flasche beschädigt war (1) oder intakt (0). Als Merkmale wurden u. a. der Mittelwert, die Standardabweichung, das Maximum und Minimum der Schwingungssignale verwendet.

### Testergebnis

Die Klassifikation wurde auf einem kleinen Testdatensatz durchgeführt. Die resultierende Confusion Matrix lautete:

| Ist / Vorhergesagt | 0 (intakt) | 1 (defekt) |
|--------------------|------------|------------|
| 0 (intakt)         | 14         | 0          |
| 1 (defekt)         | 1          | 0          |

Das Modell hat keine defekte Flasche korrekt erkannt. Die resultierenden Metriken:

- Precision (Klasse 1): 0.00  
- Recall (Klasse 1): 0.00  
- F1-Score (Klasse 1): 0.00  
- Gesamtgenauigkeit: 93 %

### Interpretation

Da im Testdatensatz nur **eine einzige defekte Flasche** enthalten war, konnte das Modell diese nicht sinnvoll als Klasse lernen. Die Datenlage ist stark unausgewogen – und solche extremen Klassenverhältnisse führen häufig zu Modellen, die nur die Mehrheitsklasse vorhersagen.

Für aussagekräftige Ergebnisse wird empfohlen:

- Zusätzliche Daten mit defekten Flaschen zu sammeln,
- oder bei Trainingsmodellen eine Gewichtung der Klassen (`class_weight='balanced'`) zu verwenden,
- optional: künstliches Oversampling der Minderheitsklasse (z. B. SMOTE).

Trotzdem zeigt das Modell bereits ein stabiles Verhalten gegenüber der intakten Klasse.
